# Second set of parameters

**maxlen = 100,
batch size = 16,
epochs = 10,
steps per epoch = 128,
embedding dimension = 64**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras

df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SU23 Deep Learning/Final Project/Yelp Restaurant Reviews.csv",index_col=0)
df=df.loc[:,["Review Text","Rating"]].dropna().reset_index(drop=True)

In [3]:
from sklearn.model_selection import train_test_split
rev=df["Review Text"]
target=df["Rating"]
x_train, x_test, y_train, y_test = train_test_split(rev,target,test_size=0.2,random_state=0,stratify=target)

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

#Text sampling
max_features = 10000
maxlen = 100
batch_size = 16

#Tokenizing
tokenizer=Tokenizer(max_features,oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)
X_train=tokenizer.texts_to_sequences(x_train)
X_test=tokenizer.texts_to_sequences(x_test)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

#Turns tockenized texts into padded sequences
print('Pad sequences (samples x text length)')
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

15916 train sequences
3980 test sequences
Pad sequences (samples x text length)
X_train shape: (15916, 100)
X_test shape: (3980, 100)


# SimpleRNN

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(16))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mae', metrics='mape') #mape: Mean Absolute Percentage Error
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 128,
                    batch_size=batch_size,
                   validation_data=(X_test,y_test))

keras.backend.clear_session()
del model

Epoch 1/10
128/128 [==============================] - 20s 129ms/step - loss: 1.4757 - mape: 44.9766 - val_loss: 0.9472 - val_mape: 40.3913
Epoch 2/10
128/128 [==============================] - 17s 131ms/step - loss: 0.9050 - mape: 40.7590 - val_loss: 0.8949 - val_mape: 44.5341
Epoch 3/10
128/128 [==============================] - 21s 167ms/step - loss: 0.8962 - mape: 47.4949 - val_loss: 0.8700 - val_mape: 46.5009
Epoch 4/10
128/128 [==============================] - 17s 131ms/step - loss: 0.8696 - mape: 45.5528 - val_loss: 0.8703 - val_mape: 46.4757
Epoch 5/10
128/128 [==============================] - 14s 112ms/step - loss: 0.8826 - mape: 47.7234 - val_loss: 0.8697 - val_mape: 46.5234
Epoch 6/10
128/128 [==============================] - 16s 126ms/step - loss: 0.8541 - mape: 47.0403 - val_loss: 0.8691 - val_mape: 46.5702
Epoch 7/10
128/128 [==============================] - 16s 126ms/step - loss: 0.8391 - mape: 45.1680 - val_loss: 0.8693 - val_mape: 46.5589
Epoch 8/10
128/128 [=======

# Pure CNN with word Embedding

In [6]:
from keras.models import Sequential
from keras.layers import Embedding
from keras import layers

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mae', metrics='mape') #mape: Mean Absolute Percentage Error
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 128,
                    batch_size=batch_size,
                   validation_data=(X_test,y_test))

keras.backend.clear_session()
del model

Epoch 1/10
128/128 [==============================] - 4s 25ms/step - loss: 1.2619 - mape: 46.9336 - val_loss: 0.8255 - val_mape: 36.1013
Epoch 2/10
128/128 [==============================] - 4s 28ms/step - loss: 0.7539 - mape: 33.7914 - val_loss: 0.7759 - val_mape: 36.8095
Epoch 3/10
128/128 [==============================] - 4s 31ms/step - loss: 0.7783 - mape: 34.9498 - val_loss: 0.8219 - val_mape: 31.6060
Epoch 4/10
128/128 [==============================] - 4s 35ms/step - loss: 0.7457 - mape: 32.1755 - val_loss: 0.7272 - val_mape: 33.2378
Epoch 5/10
128/128 [==============================] - 3s 24ms/step - loss: 0.7286 - mape: 31.9730 - val_loss: 0.7196 - val_mape: 31.5013
Epoch 6/10
128/128 [==============================] - 4s 29ms/step - loss: 0.7164 - mape: 30.4673 - val_loss: 0.7066 - val_mape: 30.7491
Epoch 7/10
128/128 [==============================] - 4s 34ms/step - loss: 0.7263 - mape: 32.8871 - val_loss: 0.7061 - val_mape: 29.5009
Epoch 8/10
128/128 [=====================

# Pure LSTM with Word Embedding

In [7]:
from keras.models import Sequential
from keras.layers import Embedding
from keras import layers

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(layers.LSTM(32,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))
model.add(layers.LSTM(16))
model.add(layers.Dense(1))
model.compile(optimizer='adam', loss='mae', metrics='mape')
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 128,
                    batch_size=batch_size,
                    validation_data=(X_test,y_test))
keras.backend.clear_session()
del model

Epoch 1/10
128/128 [==============================] - 41s 272ms/step - loss: 1.3490 - mape: 51.9830 - val_loss: 0.8800 - val_mape: 45.5841
Epoch 2/10
128/128 [==============================] - 31s 244ms/step - loss: 0.8904 - mape: 44.0250 - val_loss: 0.8682 - val_mape: 46.3033
Epoch 3/10
128/128 [==============================] - 32s 251ms/step - loss: 0.8648 - mape: 44.8720 - val_loss: 0.8665 - val_mape: 45.9601
Epoch 4/10
128/128 [==============================] - 33s 262ms/step - loss: 0.8785 - mape: 47.0805 - val_loss: 0.8502 - val_mape: 44.9482
Epoch 5/10
128/128 [==============================] - 31s 244ms/step - loss: 0.7533 - mape: 37.0548 - val_loss: 0.6189 - val_mape: 29.1306
Epoch 6/10
128/128 [==============================] - 35s 273ms/step - loss: 0.6486 - mape: 28.6944 - val_loss: 0.6074 - val_mape: 29.3928
Epoch 7/10
128/128 [==============================] - 33s 261ms/step - loss: 0.6030 - mape: 25.7103 - val_loss: 0.5915 - val_mape: 25.9186
Epoch 8/10
128/128 [=======

# Combine CNN and LSTM with Word Embedding

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D
from keras.layers import Dense

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.LSTM(32,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))
model.add(layers.LSTM(16))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mae', metrics='mape')
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 128,
                    batch_size=batch_size,
                   validation_data=(X_test,y_test))
keras.backend.clear_session()
del model

Epoch 1/10
128/128 [==============================] - 21s 111ms/step - loss: 1.3938 - mape: 50.2889 - val_loss: 0.8683 - val_mape: 46.3416
Epoch 2/10
128/128 [==============================] - 12s 93ms/step - loss: 0.8713 - mape: 44.8321 - val_loss: 0.8553 - val_mape: 45.0405
Epoch 3/10
128/128 [==============================] - 11s 87ms/step - loss: 0.7731 - mape: 38.2155 - val_loss: 0.7453 - val_mape: 39.2491
Epoch 4/10
128/128 [==============================] - 12s 95ms/step - loss: 0.6461 - mape: 30.5566 - val_loss: 0.6624 - val_mape: 24.1382
Epoch 5/10
128/128 [==============================] - 12s 96ms/step - loss: 0.5970 - mape: 26.7755 - val_loss: 0.5454 - val_mape: 24.2114
Epoch 6/10
128/128 [==============================] - 12s 96ms/step - loss: 0.5779 - mape: 25.2158 - val_loss: 0.6415 - val_mape: 32.1354
Epoch 7/10
128/128 [==============================] - 14s 112ms/step - loss: 0.5335 - mape: 22.7657 - val_loss: 0.5222 - val_mape: 22.9019
Epoch 8/10
128/128 [============

# Combine CNN with Bidirectional LSTM

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D
from keras.layers import Dense

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.Bidirectional(
    layers.LSTM(32,dropout=0.2,recurrent_dropout=0.2,return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(16)))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mae', metrics='mape')
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 128,
                    batch_size=batch_size,
                   validation_data=(X_test,y_test))
keras.backend.clear_session()
del model

Epoch 1/10
128/128 [==============================] - 33s 167ms/step - loss: 1.1782 - mape: 47.8325 - val_loss: 0.8542 - val_mape: 44.7311
Epoch 2/10
128/128 [==============================] - 20s 156ms/step - loss: 0.8280 - mape: 44.1737 - val_loss: 0.7009 - val_mape: 32.1299
Epoch 3/10
128/128 [==============================] - 19s 146ms/step - loss: 0.6677 - mape: 30.8299 - val_loss: 0.6596 - val_mape: 27.9900
Epoch 4/10
128/128 [==============================] - 19s 148ms/step - loss: 0.6097 - mape: 27.0490 - val_loss: 0.6102 - val_mape: 28.0182
Epoch 5/10
128/128 [==============================] - 25s 195ms/step - loss: 0.5684 - mape: 24.7889 - val_loss: 0.5577 - val_mape: 23.8703
Epoch 6/10
128/128 [==============================] - 20s 156ms/step - loss: 0.5453 - mape: 23.2742 - val_loss: 0.5418 - val_mape: 22.7847
Epoch 7/10
128/128 [==============================] - 19s 152ms/step - loss: 0.5352 - mape: 22.7262 - val_loss: 0.5808 - val_mape: 20.8410
Epoch 8/10
128/128 [=======